In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import os, shutil
from shutil import copyfile
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
t = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        t += 1
print(f"{t} files in dataset")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

18076 files in dataset


In [3]:
desired_width = 224
desired_height = 224
channels = 3
num_classes = 4

# Define the image size and batch size
image_size = (desired_height, desired_width)
# Define a list of possible batch sizes as per dataset size you shoud add bigger sizes
BATCH_SIZES = [64, 128, 256, 512]
# Generate a random index to select a batch size from the list
random_index = np.random.randint(0, len(BATCH_SIZES))

# Select a batch size using the random index
BATCH_SIZE = BATCH_SIZES[random_index]
classes = ["1", "2", "3", "4"]

In [4]:
# def copy_dir(source_dir, destination_dir):
#     shutil.copytree(source_dir, destination_dir)

# source_dir = "/kaggle/input/severstal-steel-defect-detection"
# destination_dir = "/kaggle/working/severstal-steel-defect-detection"

# copy_dir(source_dir, destination_dir)

In [5]:
data_path = '/kaggle/input/severstal-steel-defect-detection'
train_images_folder = os.path.join(data_path, 'train_images')
test_images_folder = os.path.join(data_path, 'test_images')

In [6]:
# Read the CSV file
df = pd.read_csv(os.path.join(data_path, "train.csv"))
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [7]:
# Define the path for the main folder
main_folder_path = '/kaggle/working/severstal-steel-defect-detection'

# Create the main folder
os.makedirs(main_folder_path, exist_ok=True)

# Define the paths for the sub-folders
train_images_folder_path = os.path.join(main_folder_path, 'train_images')
test_images_folder_path = os.path.join(main_folder_path, 'test_images')

# Create the sub-folders
os.makedirs(train_images_folder_path, exist_ok=True)
os.makedirs(test_images_folder_path, exist_ok=True)

# Optionally, you can print a message to confirm that the folders are created
print("Folders created successfully!")

Folders created successfully!


In [8]:
## create folder for each classes
# Define the main folder path
main_folder_path = '/kaggle/working/severstal-steel-defect-detection'

# Define the paths for the sub-folders
train_images_folder_path = os.path.join(main_folder_path, 'train_images')
test_images_folder_path = os.path.join(main_folder_path, 'test_images')

# Function to create sub-folders for classes in the given folder
def create_class_folders(parent_folder_path, class_list):
    for class_id in class_list:
        class_folder_path = os.path.join(parent_folder_path, class_id)
        os.makedirs(class_folder_path, exist_ok=True)

# Create sub-folders for classes in the train_images folder
create_class_folders(train_images_folder_path, classes)

# Create sub-folders for classes in the test_images folder
create_class_folders(test_images_folder_path, classes)

# Optionally, you can print a message to confirm that the folders are created
print("Folders created successfully!")

Folders created successfully!


In [9]:
# move the images to correct folders
# Define the main folder path
source_folder_path = '/kaggle/input/severstal-steel-defect-detection'

# Define the paths for the sub-folders
train_images_folder_path = os.path.join(source_folder_path, 'train_images')
# test_images_folder_path = os.path.join(source_folder_path, 'test_images')

main_folder_path = '/kaggle/working/severstal-steel-defect-detection'

# Define the paths for the sub-folders
train_images_path = os.path.join(main_folder_path, 'train_images')
test_images_path = os.path.join(main_folder_path, 'test_images')

# Copy images to the respective class sub-folders
# train
for index, row in df.iterrows():
    image_id = row['ImageId']
    class_id = str(row['ClassId'])
    source_path = os.path.join(train_images_folder_path, image_id)
    destination_path = os.path.join(train_images_path, f'{class_id}', image_id)
    shutil.copy(source_path, destination_path)


# Optionally, you can print a message to confirm that the images are copied
print("Images copied successfully!")



Images copied successfully!


In [10]:
# Extract the necessary columns
image_ids = df["ImageId"].tolist()
class_ids = df["ClassId"].tolist()
encoded_pixels = df["EncodedPixels"].tolist()

In [11]:
def preprocess_image(image, desired_width=256, desired_height=256, uni8_required=False):
    # Check if the image is empty
    if image is None:
        print("Error: Image is empty.")
        return None
    
    # Check if the image has the correct depth
    if image.dtype != np.uint8:
        print("Error: Incorrect image depth. Expected uint8.")
        return None
    
    # Resize the image to the desired size
    resized_image = cv2.resize(image, (desired_width, desired_height))
    
    # Normalize the pixel values to a specific range (e.g., 0 to 1)
    normalized_image = resized_image / 255.0
    
    # Apply any additional preprocessing steps here
    # For example, you can perform data augmentation, apply color transformations, etc.
    
    # Convert the image to the appropriate data type (e.g., float32)
    preprocessed_image = np.float32(normalized_image)

    if uni8_required:
        # Convert the image to uint8 
        preprocessed_image = cv2.convertScaleAbs(preprocessed_image * 255.0)
    
    return preprocessed_image


In [12]:
# resize one image
def resize_images(source_dir, target_dir, image_path, target_size):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, target_size)
    _, filename = os.path.split(image_path)
    save_path = os.path.join(target_dir, filename)
    cv2.imwrite(save_path, resized_image)
    return resized_image

In [13]:
# Function to resize an image
def resize_image(image_path, target_size):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, target_size)
    return resized_image

main_folder_path = '/kaggle/working/severstal-steel-defect-detection'

# Define the paths for the sub-folders
train_images_folder = os.path.join(main_folder_path, 'train_images')
test_images_folder = os.path.join(main_folder_path, 'test_images')

# Set the target size for resized images
image_size = (128, 128)

# Iterate over the test images
for image_file in os.listdir(test_images_folder):
    # Check if the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Construct the path to the image file
        image_path = os.path.join(test_images_folder, image_file)

        # Resize the image
        resized_image = resize_image(image_path, image_size)

        # Save the resized image
        resized_image_path = os.path.join(test_images_folder, f"{image_file}")
        cv2.imwrite(resized_image_path, resized_image)

print(f"Resized images saved in {test_images_folder}")


Resized images saved in /kaggle/working/severstal-steel-defect-detection/test_images


In [14]:
# Iterate over the test images
for image_file in os.listdir(test_images_folder):
    # Check if the file is an image
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Construct the path to the image file
        image_path = os.path.join(test_images_folder, image_file)

        # Resize the image
        resized_image = resize_image(image_path, image_size)

        # Save the resized image
        resized_image_path = os.path.join(test_images_folder, f"{image_file}")
        cv2.imwrite(resized_image_path, resized_image)

print(f"Resized images saved in {test_images_folder}")

Resized images saved in /kaggle/working/severstal-steel-defect-detection/test_images


In [15]:
# Split the dataset into train and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    image_ids, class_ids, test_size=0.2, random_state=42
)

In [16]:
train_images = []
test_images = []

In [17]:
# Preprocess train images
# train_images_folder = resized_images_folder
for filename in os.listdir(train_images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(train_images_folder, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image, desired_width, desired_height)
        train_images.append(preprocessed_image)

train_images = np.array(train_images)

In [18]:
# Preprocess test images
for filename in os.listdir(test_images_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_images_folder, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image, desired_width, desired_height)
        test_images.append(preprocessed_image)

test_images = np.array(test_images)

In [19]:
# Initialize dictionaries to count the number of images for each class
print(classes)
# Function to count the number of images for each class in a folder
def count_images_in_folder(folder_path, classes):
    class_counts = {class_id: 0 for class_id in classes}
    for class_id in classes:
        class_folder_path = os.path.join(folder_path, class_id)
        for image_file in os.listdir(class_folder_path):
            # Check if the file is an image
            if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                class_counts[class_id] += 1
    return class_counts

# Count the number of images for each class in the train_images folder
train_images_path = '/kaggle/working/severstal-steel-defect-detection/train_images'
train_class_counts = count_images_in_folder(train_images_path, classes)

# Print the number of images for each class
print("Train images per class:", train_class_counts)

['1', '2', '3', '4']
Train images per class: {'1': 897, '2': 247, '3': 5150, '4': 801}


In [20]:
# Set the image size
image_size = (224, 224)

# # Create the directories for the training and test sets
train_images_folder = train_images_path
test_images_folder = test_images_path

# Check for null characters in the folder names
for folder_name in [train_images_folder, test_images_folder]:
    if any(c.isspace() for c in str(folder_name)):
        raise ValueError(f"The folder names must not contain any spaces or other special characters. {folder_name}")

# Create the generators for the training and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Rescale the pixel values to the range [0, 1]
    rotation_range=20,   # Randomly rotate the images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift the images horizontally by up to 20% of the image width
    height_shift_range=0.2, # Randomly shift the images vertically by up to 20% of the image height
    shear_range=0.2,         # Randomly shear the images by up to 20%
    zoom_range=0.3,          # Randomly zoom the images by up to 20%
    horizontal_flip=True,    # Randomly flip the images horizontally
    vertical_flip=True,
    brightness_range = (0.8, 1.2), # brightness adjuster so that some partial images can detect when ground is dark
    fill_mode='nearest',      # Fill any empty pixels with the nearest available pixel value
    featurewise_center=False,
    featurewise_std_normalization=False,
    data_format='channels_last',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255   # Rescale the pixel values to the range [0, 1]
)

# Normalize the data
train_datagen.mean = [123.68, 116.779, 103.939] # RGB channel mean values from ImageNet
train_datagen.std = [58.393, 57.12, 57.375]

# Load the training data into x_train and y_train
train_generator = train_datagen.flow_from_directory(
    directory=train_images_folder,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True  # Shuffle the files from different folders
)

# Load the validation data into x_val and y_val
validation_generator = train_datagen.flow_from_directory(
    directory=train_images_folder,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the test data into x_test and y_test
test_generator = test_datagen.flow_from_directory(
    directory=test_images_folder,
    target_size=image_size,
    batch_size=16,
    class_mode='categorical'
)

# Handle any exceptions that may occur during data generation
try:
    x_train, y_train = next(train_generator)
except Exception as e:
    print(f"Error during training data generation: {e}")
    
try:
    x_val, y_val = next(validation_generator)
except Exception as e:
    print(f"Error during validation data generation: {e}")

try:
    x_test, y_test = next(test_generator)
except Exception as e:
    print(f"Error during test data generation: {e}")


Found 5677 images belonging to 4 classes.
Found 1418 images belonging to 4 classes.
Found 0 images belonging to 4 classes.


In [21]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(desired_height, desired_width, channels))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [22]:
# Create a new model and add the pre-trained VGG16 base
model = Sequential()
model.add(base_model)

# Add custom layers for classification
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [23]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 6,423,812
Non-trainable params: 14,714,688
_________________________________________________________________


In [24]:
# Define the target validation accuracy (75% in this case)
target_accuracy = 0.75

# Calculate the min_delta value to stop training at the target accuracy
min_delta = target_accuracy - 1.0

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, mode='max', min_delta=min_delta, verbose=1)

In [25]:
# Train the model with early stopping
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/100
178/178 [==============================] - 151s 781ms/step - loss: 0.7593 - accuracy: 0.7305 - val_loss: 0.6917 - val_accuracy: 0.7419
Epoch 2/100
178/178 [==============================] - 135s 757ms/step - loss: 0.6549 - accuracy: 0.7476 - val_loss: 0.6396 - val_accuracy: 0.7482
Epoch 3/100
178/178 [==============================] - 135s 758ms/step - loss: 0.6192 - accuracy: 0.7611 - val_loss: 0.6834 - val_accuracy: 0.7334
Epoch 4/100
178/178 [==============================] - 134s 754ms/step - loss: 0.6043 - accuracy: 0.7581 - val_loss: 0.5929 - val_accuracy: 0.7525
Epoch 5/100
178/178 [==============================] - 135s 756ms/step - loss: 0.5836 - accuracy: 0.7714 - val_loss: 0.5815 - val_accuracy: 0.7708
Epoch 6/100
178/178 [==============================] - 134s 750ms/step - loss: 0.5832 - accuracy: 0.7668 - val_loss: 0.5867 - val_accuracy: 0.7560
Epoch 6: early stopping


In [26]:
# Check if the desired accuracy and validation accuracy thresholds are met
if history.history['accuracy'][-1] >= 0.75 and history.history['val_accuracy'][-1] >= 0.7:
    print("Early stopping criteria met!")

Early stopping criteria met!


In [35]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Create an empty list to store the results
submission_data = []

data_path = '/kaggle/input/severstal-steel-defect-detection'
test_images_folder = os.path.join(data_path, 'test_images')

# Iterate over the test images
for image_file in os.listdir(test_images_folder):
    # Load and preprocess the test image
    image_path = os.path.join(test_images_folder, image_file)
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        test_image = load_img(image_path, target_size=(224, 224))
        test_image = img_to_array(test_image)
        test_image = preprocess_input(test_image)
        test_image = np.expand_dims(test_image, axis=0)
        
        # Make predictions on the test image
        predictions = model.predict(test_image)
        
        # Get the predicted class label
        predicted_class_label = np.argmax(predictions)
        
        # Append the result to the submission_data list
        submission_data.append((image_file, f"1 409600", predicted_class_label))

# Create a DataFrame from the submission_data list
submission_df = pd.DataFrame(submission_data, columns=["ImageId", "EncodedPixels", "ClassId"])

# Save the DataFrame to a CSV file
submission_df.to_csv("submission.csv", index=False)


1/1 [==============================] - 0s 24ms/step


In [36]:
sub_df = pd.read_csv("submission.csv")
sub_df.head()

,ImageId,EncodedPixels,ClassId
0,6d2a4005f.jpg,1 409600,3
1,a2491cf5f.jpg,1 409600,2
2,0854a9de6.jpg,1 409600,3
3,6bda73259.jpg,1 409600,2
4,ba8ebb757.jpg,1 409600,2
